In [28]:
import pandas as pd

In [29]:
DATA_PATH = "./SentencePuzzleKD/KD_train_gpt-4_revised.csv"

In [30]:
df = pd.read_csv(DATA_PATH)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507 entries, 0 to 506
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            507 non-null    object
 1   question      507 non-null    object
 2   option_1      507 non-null    object
 3   hypothesis_1  507 non-null    object
 4   option_2      507 non-null    object
 5   hypothesis_2  507 non-null    object
 6   option_3      507 non-null    object
 7   hypothesis_3  507 non-null    object
 8   option_4      507 non-null    object
 9   answer        507 non-null    object
 10  label         507 non-null    int64 
 11  STATUS        505 non-null    object
 12  Hard          53 non-null     object
dtypes: int64(1), object(12)
memory usage: 51.6+ KB


In [32]:
def annotate_sample(id, q, a, h):
    return {"id": id, "question": q, "answer": a, "hypothesis": h}

In [33]:
def annotate_samples(df):
    samples = []
    for i, row in df.iterrows():
        answer_key = row["label"]
        option_key = f"option_{answer_key}"
        hypo_key = f"hypothesis_{answer_key}"
        if row[option_key] != row["answer"]:
            print("Error: option_key and answer_key do not match")
            return 
        id = row["id"]
        question = row["question"]
        answer = row[option_key]
        if answer_key == 4:
            hypothesis = f"The answer to the question is not in one of the options: [{row['option_1']}, {row['option_2']}, {row['option_3']}]"
        else:
            hypothesis = row[hypo_key]
        samples.append(annotate_sample(id, question, answer, hypothesis))
    return samples

In [34]:
samples = annotate_samples(df)

In [36]:
samples_df = pd.DataFrame(samples)

In [37]:
samples_df

,id,question,answer,hypothesis
0,SP-0,Mr. and Mrs. Mustard have six daughters and ea...,Each daughter shares the same brother.,The question states that Mr. and Mrs. Mustard ...
1,SP-0_SR,The six daughters of Mr. and Mrs. Mustard each...,Each daughter shares the same brother.,The only way that what question asked is possi...
2,SP-0_CR,"A chess team has five players, and each player...",Each player shares the same coach.,The only way this could be possible is if the...
3,SP-1,A woman shoots her husband. Then she holds him...,The woman was a photographer. She shot a pictu...,The question is a riddle that uses misleading ...
4,SP-1_SR,An individual shoots their spouse. She continu...,The woman was a photographer. She shot a pictu...,The question describes a series of actions tha...
...,...,...,...,...
502,SP-207_SR,"Ten pears hung high, ten men passed past, Each...",EACH is the name of one of the men.,The question presents a riddle that seems to d...
503,SP-207_CR,There were twenty boys with different names in...,All is the name of one of the boys.,The question presents a scenario where twenty ...
504,SP-208,"The more you take, the more you leave behind",Footsteps.,"The question ""The more you take, the more you ..."
505,SP-208_SR,"The more you take, the more you abandon.",Footsteps.,The question is a riddle that requires a metap...


In [38]:
import os
if os.path.exists("./rag"):
    print("Directory exists")
else:
    os.mkdir("./rag")

In [42]:
filename = os.path.basename(DATA_PATH)
file_splited = filename.split("_")
split_mode = file_splited[1]
model_name = file_splited[2]


In [44]:
samples_df.to_csv(f"./rag/{split_mode}_{model_name}_raw.csv", index=False)